In [62]:
import numpy as np
import sys 
import os
sys.path.append('/home/giorgiod/Documents/APDFT/dep')
sys.path.append('/home/giorgiod/Documents/APDFT/src')

In [63]:
from apdft import Calculator

In [64]:
calc_path = '/home/giorgiod/MRCC_interface/Diatomics/from_nn_opt_ccsd_ccpvtz_77_to_68_A5_G5/A_2_G_0/'
proj_path= '/home/giorgiod/MRCC_interface/Diatomics/from_nn_opt_ccsd_ccpvtz_77_to_68_A5_G5/'
grid = Calculator.MrccCalculator.get_grid(None, None, calc_path)

In [65]:
print (type(grid))
grid[0].shape, grid[1].shape #grid=(array[[x0,y0,z0],...,[xN,yN,zN]], array[w0,...wN])

<class 'tuple'>


((58032, 3), (58032,))

In [66]:
ccdensity = Calculator.MrccCalculator.density_on_grid(calc_path + 'DENSITY', grid[0])

In [67]:
(ccdensity * grid[1]).sum()

13.999999245167064

In [68]:
Calculator.MrccCalculator.get_total_energy(calc_path)

{"filename": "/home/giorgiod/MRCC_interface/Diatomics/from_nn_opt_ccsd_ccpvtz_77_to_68_A5_G5/A_2_G_0//run.log", "level": "error", "event": "Unable to read energy from log file.", "timestamp": "2019-06-12 14:10"}


0

In [69]:
def parse_energy_cc(log_file):
    """Parse the couple cluster energy from an MRCC output file"""
    with open(log_file,'r') as logf:
        while True:
            line=logf.readline()
            if "Final results:" in line:
                good_line=logf.readline()
                if "Total CCSD energy" in good_line:
                    for x in good_line.split(' '):
                        try:
                            float(x)
                            return (float(x))
                        except:
                            pass    

In [70]:
gl=Calculator.MrccCalculator.parse_energy_cc_Mrcc(calc_path+'/run.log')
print(gl)

-109.381056423703


In [71]:
from apdft.physics import Coulomb

In [72]:
def get_energy(proj_path,ao,go):
    return (parse_energy_cc("{}A_{}_G_{}/run.log".format(proj_path,ao,go)))

In [73]:
get_energy(proj_path,2,1)

-109.379978610182

In [74]:
dX= 0.03779452267842504  # bohrs =.02 angstrom
dL=0.05  ## delta Lambda = 0.05

In [75]:
k= (get_energy(proj_path,0,1)+get_energy(proj_path,0,-1)-2*get_energy(proj_path,0,0))/dX**2

In [76]:
#### constants #####
angstorm=1.889726133921252
N_mass=25532.936684486547
centimeter=188972613.39212522
light_speed= 137.03599975303575
C_mass=21893.98614768941
O_mass=29165.124304476205
mu_nn=N_mass/2

In [77]:
def to_cm(k,mu=mu_nn):
    return (k/mu)**0.5*centimeter/light_speed/(2*np.pi)

In [78]:
to_cm(k) ### ok good job

2445.175259775592

In [79]:
def common_grid(go):
    return (Calculator.MrccCalculator.get_grid(None, None, proj_path+'A_0_G_{}/'.format(go)))

In [80]:
####### alchemy expantion ####  COMMON_GRID IS TO BE DEFINED FOR EACH GEOMETRY  
def get_ccdensity(ao,go):
    return Calculator.MrccCalculator.density_on_grid("{}A_{}_G_{}/DENSITY".format(proj_path,ao,go),common_grid(go)[0])

In [81]:
get_ccdensity(-2,-2)-get_ccdensity(2,-2)

array([0., 0., 0., ..., 0., 0., 0.])

In [82]:
get_energy(proj_path,0,1)

-109.379978610182

In [83]:
from apdft.physics import Coulomb

In [84]:
### very poor code 
nn_minR=1.093635097483
atoms=np.asarray([7.,7.])
def coordinates(go):
    return np.asarray([[0.,0.,0.],[ nn_minR+go*dX,0.,0.]])
target=np.asarray([6.,8.])

In [85]:
print(Coulomb.nuclei_nuclei(coordinates(0),atoms)/49)
print(Coulomb.nuclei_nuclei(coordinates(0),target)/48)

0.4838699963890157
0.4838699963890157


In [90]:
from apdft.physics import angstrom

In [100]:
ds = []
for site in coordinates(0):
    ds.append(np.linalg.norm((common_grid(0)[0] - site)*angstrom,axis=1))

# get target predictions

deltaV = np.zeros(len(common_grid(0)[1]))
for atomidx in range(len(atoms)):
    deltaV += (target[atomidx]-atoms[atomidx]) / ds[atomidx]


$\tilde{\rho} = \rho^0 +\frac{1}{2}\rho' +\frac{1}{6}\rho " + \frac{1}{24}\rho ^{(3)}+ \frac{1}{120}\rho ^{(4)} $ 

## 5point coefficients <br>
0  :      0    :  1    :  0   :     0  <br>
1/12 :	−2/3 :	0 	 : 2/3 :	−1/12    for first deriv <br>
−1/12 :	4/3 :	−5/2 : 4/3 : −1/12  for second deriv <br>
−1/2 :	 1 	:    0 	:  −1    :  1/2   for third deriv <br>
 1  :	−4  :	6 	:  −4   : 1       for fourth deriv <br>
 
 

In [122]:
coeff0=np.asarray([0,0,1,0,0])
coeff1=np.asarray([1/12,-2/3,0,2/3,-1/12])
coeff2=np.asarray([-1/12 , 4/3 , -5/2 , 4/3 , -1/12])
coeff3=np.asarray([-1/2 , 1 , 0 , -1 , 1/2])
coeff4=np.asarray([1 ,-4 , 6 ,-4 , 1])
coeff_tilda=coeff0+1/2*coeff1+1/6*coeff2+1/24*coeff3+1/120*coeff4
print (coeff_tilda, np.sum(coeff_tilda))
coeff_taylor=coeff0+coeff1+1/2*coeff2+1/26*coeff3+1/24*coeff4
print (coeff_taylor, np.sum(coeff_taylor))

[ 0.01527778 -0.10277778  0.63333333  0.48055556 -0.02638889] 1.0000000000000002
[ 0.06410256 -0.12820513  0.          1.12820513 -0.06410256] 0.9999999999999998


SyntaxError: invalid syntax (<ipython-input-107-f71d4dee1edb>, line 2)